In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()

In [3]:
# Using regex to find text in website facebook example
browser = webdriver.Chrome()
browser.get("https://www.facebook.com/")
forgotAccountLink = browser.find_element(By.XPATH, "//*[text()='Forgot Password?']")
forgotAccountLink.get_attribute('innerHTML')
driver.close()

'Forgot Password?'

In [4]:
browser.get("http://image-net.org/archive/words.txt")
shapeLink = browser.find_element(By.XPATH, "//*[text()='shape']")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[text()='shape']"}
  (Session info: chrome=87.0.4280.141)
